# This Script Explores Using Stochastic Gradient Descent For Multivariate Logistic Regression

# Logistic Regression Recap

### Predictor Function:

* $\hat{y}(z) = \frac{1}{1+e^{-z}}$

    * Where $z = b + w_1x_1 + w_2x_2 + ... w_nx_n$

Because this data set has 7 input variables, our z term becomes:
* $z = b + w_1x_1 + w_2x_2 + w_3x_3 + w_4x_4 + w_5x_5 + + w_6x_6 + w_7x_7$

We can represent the repeated $w_nx_n$ term in our z term by a matrix multiplication:

* $z = b + W^TX$ (where W is a row vector of our weights and X is a row vector of our input variables)

### Cost Function:

Since we will be performing Stochastic Gradient Descent, Out Single Sample Cost Function is:

* $ Error =  -y_i \ln(\hat{y}_i) - (1-y_i) \ln(1-\hat{y}_i) $

The partial derivative of this cost function for each sample w.r.t each weight is:

* $\frac{\partial Error}{\partial w_n} (\hat{y_i}-y_i)x_n$

The partial derivative of this cost function for each sample w.r.t the bias term is:

* $\frac{\partial Error}{\partial b} (\hat{y_i}-y_i)$

### Stochastic Gradient Descent

When performing SGD, we will compute the partial cost for all weights & the bias term after pulling each
sample from the training data. We update each parameter by:

* $parameter_{new} = parameter_{old} - LearningRate*\frac{\partial Error}{\partial parameter_{old}}$

Parameters are updated n number of times each epoch where n is the size of our # of training samples


In [11]:
#making necessary imports & pulling data from csv

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

x_df = pd.read_csv('test_X_titanic.csv')
y_df = pd.read_csv('test_Y_titanic.csv')

x_train_data = x_df.values[:,1:]
y_train_data = y_df.values[:,1:]

#print(x_train_data)
#print(y_train_data)

#combining the training data arrays into 1 np array
train_data = np.append(x_train_data, y_train_data, axis=1)

print(train_data)

[[ 3.      0.     34.5    ...  7.8292  2.      0.    ]
 [ 3.      1.     47.     ...  7.      1.      1.    ]
 [ 2.      0.     62.     ...  9.6875  2.      0.    ]
 ...
 [ 3.      0.     38.5    ...  7.25    1.      0.    ]
 [ 3.      0.         nan ...  8.05    1.      0.    ]
 [ 3.      0.         nan ... 22.3583  0.      0.    ]]


In [3]:
def predictor_func(bias,weight_vec,input_var_vec):
    """
    This function returns our logistic model prediction provided our bias term, and vectors for our weights & their associated input variables
    """
    z = bias + np.matmul(weight_vec.T,input_var_vec)
    return 1/(1+np.exp(-z))

In [4]:
def partial_error_wrt_weights(y_observation,bias,weight_vec,input_var_vec):
    """
    This function returns the partial error derivative w.r.t each weight in our weight vector
    """
    #pulling our y prediction on the passed input variable vector
    y_prediction = predictor_func(bias,weight_vec,input_var_vec)
    #returning an np array of the partial associated for each weight
    return np.array([(y_prediction - y_observation)*input_var for input_var in input_var_vec])

In [6]:
def partial_error_wrt_bias(y_observation,bias,weight_vec,input_var_vec):
    """
    This function returns the partial error derivative w.r.t to the bias term
    """
    return predictor_func(bias,weight_vec,input_var_vec) - y_observation

In [7]:
def update_params(y_observation,bias,weight_vec,input_var_vec,learn_rate):
    """
    This function updates our weight parameter and bias term
    """
    #updating weights
    weight_vec_new = weight_vec - learn_rate*partial_error_wrt_weights(y_observation,bias,weight_vec,input_var_vec)
    bias_new = bias - learn_rate*partial_error_wrt_bias(y_observation,bias,weight_vec,input_var_vec)
    return weight_vec_new,bias_new

In [9]:
def error_func(training_data,bias,weight_vec):
    """
    This function computes our average error over all samples in the training data
    sample[0:6] = the x input array
    sample[-1] = the y observation
    """
    return 1/len(train_data.shape[1])*sum([-data_sample[-1]*np.log(predictor_func(bias,weight_vec,data_sample[:,6])) -
                                (1-data_sample[-1])*np.log(1-predictor_func(bias,weight_vec,data_sample[:,6])) for data_sample in training_data])

In [ ]:
#performing the SGD

#hard coding a learn_rate & epoch #
learning_rate = 0.001
epochs = 5000

#randomly assigning weight/bias term values
weights = np.array([np.random.randint(-1,1) for _ in range(x_train_data.shape[1])])
b = 1

#init logs of variables of interest

#weights
w1_log, w2_log, w3_log, w4_log, w5_log, w6_log, w7_log  = [0]*epochs,[0]*epochs,[0]*epochs,[0]*epochs,[0]*epochs,[0]*epochs,[0]*epochs
#bias
b_log = [0]*epochs
#weight partials
partial_w1_log,partial_w2_log,partial_w3_log,partial_w4_log,partial_w5_log,partial_w6_log,partial_w7_log = [0]*epochs,[0]*epochs,[0]*epochs,[0]*epochs,[0]*epochs,[0]*epochs,[0]*epochs
#bias partial
partial_b_log = [0]*epochs
#error log
error_log = [0]*epochs


for epoch in range(epochs):
    #randomly shuffling the training data
    np.random.shuffle(train_data)
    #iterating over each sample in training data and updating params
    for sample in train_data:
        #splitting our sample into the vector of x inputs and scalar y output
        x_data,y_datapoint = sample[0:6] , sample[-1]
        #finding partials for current train data sample
        weight_partials = partial_error_wrt_weights(y_datapoint,b,weights,x_data)
        bias_partial =  partial_error_wrt_bias(y_datapoint,b,weights,x_data)
        #logging partial values
        partial_w1_log[epoch] ,partial_w2_log[epoch] ,partial_w3_log[epoch],partial_w4_log[epoch],partial_w5_log[epoch],partial_w6_log[epoch],partial_w7_log[epoch]  = weight_partials
        partial_b_log[epoch] = bias_partial
        #updating our parameters
        weights,b = update_params(y_datapoint,b,weights,x_data,learning_rate)
        #logging updated parameter values
        w1_log[epoch], w2_log[epoch], w3_log[epoch], w4_log[epoch], w5_log[epoch], w6_log[epoch], w7_log[epoch] = weights
        b_log[epoch] = b
    #calculating & logging our overall Log Loss Error after each training epoch
    error_log[epoch] = error_func(train_data,b,weights)

In [ ]:
#plotting weight 1 value vs # of epochs
plt.plot(np.arange(epochs),w1_log)
plt.xlabel('Epoch #')
plt.ylabel('Weight 1 Value')
plt.title('Weight 1 Converges to its Optimal Value')
plt.show()

In [ ]:
#plotting weight 2 value vs # of epochs

plt.plot(np.arange(epochs),w2_log)
plt.xlabel('Epoch #')
plt.ylabel('Weight 2 Value')
plt.title('Weight 2 Converges to its Optimal Value')
plt.show()

In [ ]:
#plotting weight 3 value vs # of epochs

plt.plot(np.arange(epochs),w3_log)
plt.xlabel('Epoch #')
plt.ylabel('Weight 3 Value')
plt.title('Weight 3 Converges to its Optimal Value')
plt.show()

In [ ]:
#plotting weight 4 value vs # of epochs

plt.plot(np.arange(epochs),w4_log)
plt.xlabel('Epoch #')
plt.ylabel('Weight 4 Value')
plt.title('Weight 4 Converges to its Optimal Value')
plt.show()

In [ ]:
#plotting weight 5 value vs # of epochs

plt.plot(np.arange(epochs),w5_log)
plt.xlabel('Epoch #')
plt.ylabel('Weight 5 Value')
plt.title('Weight 5 Converges to its Optimal Value')
plt.show()


In [ ]:
#plotting weight 6 value vs # of epochs

plt.plot(np.arange(epochs),w6_log)
plt.xlabel('Epoch #')
plt.ylabel('Weight 6 Value')
plt.title('Weight 6 Converges to its Optimal Value')
plt.show()

In [ ]:
#plotting weight 7 value vs # of epochs

plt.plot(np.arange(epochs),w7_log)
plt.xlabel('Epoch #')
plt.ylabel('Weight 7 Value')
plt.title('Weight 7 Converges to its Optimal Value')
plt.show()


In [ ]:
#plotting Partial Derivative of Cost wrt Weight 1 vs # of epochs

plt.plot(np.arange(epochs),partial_w1_log)
plt.xlabel('Epoch #')
plt.ylabel('Partial Cost wrt Weight 1')
plt.title('Partial Cost wrt. Weight 1 Converges to 0')
plt.show()

In [ ]:
#plotting Partial Derivative of Cost wrt Weight 2 vs # of epochs

plt.plot(np.arange(epochs),partial_w2_log)
plt.xlabel('Epoch #')
plt.ylabel('Partial Cost wrt Weight 2')
plt.title('Partial Cost wrt. Weight 2 Converges to 0')
plt.show()

In [ ]:
#plotting Partial Derivative of Cost wrt Weight 3 vs # of epochs

plt.plot(np.arange(epochs),partial_w3_log)
plt.xlabel('Epoch #')
plt.ylabel('Partial Cost wrt Weight 3')
plt.title('Partial Cost wrt. Weight 3 Converges to 0')
plt.show()


In [ ]:
#plotting Partial Derivative of Cost wrt Weight 4 vs # of epochs

plt.plot(np.arange(epochs),partial_w4_log)
plt.xlabel('Epoch #')
plt.ylabel('Partial Cost wrt Weight 4')
plt.title('Partial Cost wrt. Weight 4 Converges to 0')
plt.show()



In [ ]:
#plotting Partial Derivative of Cost wrt Weight 5 vs # of epochs

plt.plot(np.arange(epochs),partial_w5_log)
plt.xlabel('Epoch #')
plt.ylabel('Partial Cost wrt Weight 5')
plt.title('Partial Cost wrt. Weight 5 Converges to 0')
plt.show()


In [ ]:
#plotting Partial Derivative of Cost wrt Weight 6 vs # of epochs

plt.plot(np.arange(epochs),partial_w6_log)
plt.xlabel('Epoch #')
plt.ylabel('Partial Cost wrt Weight 6')
plt.title('Partial Cost wrt. Weight 6 Converges to 0')
plt.show()

In [ ]:
#plotting Partial Derivative of Cost wrt Weight 7 vs # of epochs

plt.plot(np.arange(epochs),partial_w7_log)
plt.xlabel('Epoch #')
plt.ylabel('Partial Cost wrt Weight 7')
plt.title('Partial Cost wrt. Weight 7 Converges to 0')
plt.show()


In [ ]:
#plotting bias term value vs # of epochs

plt.plot(np.arange(epochs),b_log)
plt.xlabel('Epoch #')
plt.ylabel('Bias Term Value')
plt.title('Bias Term Converges to its Optimal Value')
plt.show()

In [ ]:
#plotting Partial Derivative of Cost wrt Bias Term vs # of epochs

plt.plot(np.arange(epochs),partial_b_log)
plt.xlabel('Epoch #')
plt.ylabel('Partial Cost wrt Bias Term')
plt.title('Partial Cost wrt. Bias Term Converges to 0')
plt.show()

In [ ]:
#plotting Cost vs # of epochs

plt.plot(np.arange(epochs),error_log)
plt.xlabel('Epoch #')
plt.ylabel('Cost Value')
plt.title('Total Cost Minimizes under Gradient Descent ')
plt.show()

In [ ]:
#calculating model accuracy with optimal parameters
opt_weights = w1_log[-1],w2_log[-1],w3_log[-1],w4_log[-1],w5_log[-1],w6_log[-1],w7_log[-1]
opt_b = b_log[-1]
#init counter of right predictions
right_predicts = 0

#Calculating our model accuracy against the training data
for row_index in range(train_data.shape[0]):
    observation_vec = row_index[row_index,:]
    x_input_vec = observation_vec[0:6]
    y_output = observation_vec[-1]
    prediction = predictor_func(opt_b,opt_weights,x_input_vec)
    if y_output == 1 and 0.5<=prediction<=1:
        right_predicts+=1
    elif y_output == 0 and 0<=prediction<0.5:
        right_predicts+=1
    else:
        pass
print(f'This Logistic Regression Model has an overall accuracy of {round(right_predicts/train_data.shape[0])*100,4}%')
#printing optimized model params
print(f'Optimized Model Params: {opt_weights}')